# Task 1 - Preguntas Teóricas
1. ¿Por qué el modelo de Naive Bayes se le considera “naive”?

Se consideran Naive debido a que en ellos se asume que las variables predictoras son independientes entre sí. Es decir, que la presencia de una cierta característica en un conjunto de datos no está en absoluto relacionada con la presencia de cualquier otra característica.

Victor Roman. (2019). Algoritmos Naive Bayes: Fundamentos e Implementación. Medium. https://medium.com/datos-y-ciencia/algoritmos-naive-bayes-fudamentos-e-implementaci%C3%B3n-4bcb24b307f


2. Explique la formulación matemática que se busca optimizar en Support Vector Machine, además responda ¿cómo funciona el truco del Kernel para este modelo? (Lo que se espera de esta pregunta es que puedan explicar en sus propias palabras la fórmula a la que llegamos que debemos optimizar de SVM en clase)
3. 

4. Investigue sobre Random Forest y responda
    a. ¿Qué tipo de ensemble learning es este modelo?
      Pertenece al grupo de Bagging (Bootstrap Aggregating) ya que divide la serie de datos en varios subconjuntos compuestos aleatoriamente de muestras, y se entrena un modelo en cada subconjunto, esto significa que habrá tantos modelos como subconjuntos creados. Luego se combinan todos los resultados de los modelos para obtener un resultado final.
    b. ¿Cuál es la idea general detrás de Random Forest?
        Construye un conjunto diverso de árboles de decisión, donde cada uno de estos árboles se entrena en una muestra aleatoria del conjunto de datos. Un tercio se reserva como datos de prueba y a esto se le conoce como muestra fuera de la empresa (oob). Luego, se inyecta otra instancia de aleatoriedad a través del agrupamiento de características, lo que agrega más diversidad al conjunto de datos y reduce la correlación entre los árboles de decisión. La determinación de la predicción depende del tipo del problema, para una tarea de regresión se promedia los árboles de decisión individuales, mientras que para un problema de clasificación se utiliza un voto mayoritario, es decir, la variable categórica más frecuente, arrojará la clase predicha.
    c. ¿Por qué se busca baja correlación entre los árboles de Random Forest?
        La baja correlación reduce el sobreajuste y favorece la precisión del modelo, ya que asegura que cada árbol creado aporte información única. Ya que como el nombre del modelo lo indica (naive), se asume que las variables son independientes entre sí.

¿Qué es un bosque aleatorio? (s/f). Ibm.com. Recuperado el 25 de enero de 2024, de https://www.ibm.com/mx-es/topics/random-forest

Random Forest: Bosque aleatorio. Definición y funcionamiento. (2022, enero 25). Formación en ciencia de datos | Datascientest.com. https://datascientest.com/es/random-forest-bosque-aleatorio-definicion-y-funcionamiento

# Task 2 - Naive Bayes: Clasificador de Mensajes Ham/Spam
## Task 2.1 - Lectura y limpieza del dataset

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

C:\Users\aleaz\AppData\Local\Temp\ipykernel_18916\2200693214.py:3: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
file = open('entrenamiento.txt', 'r')
dataset = []
for line in file:
    print(line.split('    '))
    dataset.append(line.split('    '))

['ham', '"Go until jurong point, crazy.. Available only in bugis n great world la e buffet... Cine there got amore wat...",,,\n']
['ham', 'Ok lar... Joking wif u oni...,,,\n']
['spam', "Free entry in 2 a wkly comp to win FA Cup final tkts 21st May 2005. Text FA to 87121 to receive entry question(std txt rate)T&C's apply 08452810075over18's,,,\n"]
['ham', 'U dun say so early hor... U c already then say...,,,\n']
['ham', '"Nah I don\'t think he goes to usf, he lives around here though",,,\n']
['spam', '"FreeMsg Hey there darling it\'s been 3 week\'s now and no word back! I\'d like some fun you up for it still? Tb ok! XxX std chgs to send, ï¿½1.50 to rcv",,,\n']
['ham', 'Even my brother is not like to speak with me. They treat me like aids patent.,,,\n']
['ham', "As per your request 'Melle Melle (Oru Minnaminunginte Nurungu Vettam)' has been set as your callertune for all Callers. Press *9 to copy your friends Callertune,,,\n"]
['spam', 'WINNER!! As a valued network customer you have been

In [3]:
dataset = pd.DataFrame(dataset)
dataset

,0,1
0,ham,"""Go until jurong point, crazy.. Available only..."
1,ham,"Ok lar... Joking wif u oni...,,,\n"
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"""Nah I don't think he goes to usf, he lives ar..."
5,spam,"""FreeMsg Hey there darling it's been 3 week's ..."
6,ham,Even my brother is not like to speak with me. ...
7,ham,As per your request 'Melle Melle (Oru Minnamin...
8,spam,WINNER!! As a valued network customer you have...
9,spam,Had your mobile 11 months or more? U R entitle...


### Limpiar el dataset
- Limpiar los caracteres especiales
- Misma nomenclatura (todo mayúsculas o minúsculas)

In [4]:
# Limpiar caracteres especiales

for line in dataset.values:
    line[1] = line[1].lower()
    for c in line[1]:
        if c.isalpha() == False and c != ' ':
            line[1] = line[1].replace(c, '')
        
    line[1] = ' '.join([p for p in line[1].split(' ') if p != '']) # Quitar espacios en blanco

dataset

,0,1
0,ham,go until jurong point crazy available only in ...
1,ham,ok lar joking wif u oni
2,spam,free entry in a wkly comp to win fa cup final ...
3,ham,u dun say so early hor u c already then say
4,ham,nah i dont think he goes to usf he lives aroun...
5,spam,freemsg hey there darling its been weeks now a...
6,ham,even my brother is not like to speak with me t...
7,ham,as per your request melle melle oru minnaminun...
8,spam,winner as a valued network customer you have b...
9,spam,had your mobile months or more u r entitled to...


In [5]:
# Codificamos la variable categoricas en 1 = ham y 0 = spam  
def codif_y_ligar(dataframe_original, variables_por_codificar):
    """Codifica las variables categóricas y las une al dataframe original"""
    dummies = pd.get_dummies(dataframe_original[[variables_por_codificar]])
    resultado = pd.concat([dataframe_original, dummies], axis = 1)
    resultado = resultado.drop([variables_por_codificar], axis = 1)
    return resultado

variables_a_codificar = [0] # Variables categóricas. Status tiene valores 'spam' y 'ham'
for variable in variables_a_codificar:
    dataset = codif_y_ligar(dataset, variable)

In [6]:
X = dataset.iloc[:, 0].values
y = dataset.iloc[:, 1].values

In [7]:
y # HAM ES TRUE

array([ True,  True, False,  True,  True, False,  True,  True, False,
       False,  True, False, False,  True,  True])

In [8]:
# separar data en datos de entrenamiento y datos de prueba
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20, random_state = 0)

In [9]:
X_train

array(['had your mobile months or more u r entitled to update to the latest colour mobiles with camera for free call the mobile update co free on',
       'i have a date on sunday with will',
       'nah i dont think he goes to usf he lives around here though',
       'free entry in a wkly comp to win fa cup final tkts st may text fa to to receive entry questionstd txt ratetcs apply overs',
       'ive been searching for the right words to thank you for this breather i promise i wont take your help for granted and will fulfil my promise you have been wonderful and a blessing at all times',
       'im gonna be home soon and i dont want to talk about this stuff anymore tonight k ive cried enough today',
       'as per your request melle melle oru minnaminunginte nurungu vettam has been set as your callertune for all callers press to copy your friends callertune',
       'six chances to win cash from to pounds txt csh and send to cost pday days tsandcs apply reply hl info',
       'u dun 

## Task 2.2 - Construcción del modelo

In [10]:
lineas_ham = 0
lineas_spam = 0
for line in X_train:
    # compare to y_train and count the amount of times that the line is ham or spam
    # if ham, add 1 to ham
    # if spam, add 1 to spam
    if y_train[X_train.tolist().index(line)] == 1:
        lineas_ham += 1
    else:
        lineas_spam += 1
        
print(f"ham: {lineas_ham}, spam: {lineas_spam}")

ham: 7, spam: 5


In [11]:
T = len(X_train)
dominio = [0, 1]
K = 50

In [12]:
probabilidad_spam = (lineas_spam + K) / (T + K * len(dominio))
probabilidad_ham = (lineas_ham + K) / (T + K * len(dominio))

print(f"p_spam: {probabilidad_spam}, p_ham: {probabilidad_ham}")

p_spam: 0.49107142857142855, p_ham: 0.5089285714285714


In [13]:
diccionario_contador = dict[str, [int, int]]() # [ham, spam]

for line in X_train:
    # count the amount of times that each word appears in ham and spam
    # if the word is not in the dictionary, add it and add 1 to the ham or spam counter
    # if the word is in the dictionary, add 1 to the ham or spam counter
    for word in line.split(' '):
        if word not in diccionario_contador:
            diccionario_contador[word] = [0, 0]
        if y_train[X_train.tolist().index(line)] == 1:
            diccionario_contador[word][0] += 1 # ham
        else:
            diccionario_contador[word][1] += 1 # spam
            
print(diccionario_contador)

{'had': [0, 1], 'your': [4, 1], 'mobile': [0, 2], 'months': [0, 1], 'or': [0, 1], 'more': [0, 1], 'u': [2, 1], 'r': [0, 1], 'entitled': [0, 1], 'to': [4, 11], 'update': [0, 2], 'the': [1, 3], 'latest': [0, 1], 'colour': [0, 1], 'mobiles': [0, 1], 'with': [1, 1], 'camera': [0, 1], 'for': [4, 2], 'free': [0, 4], 'call': [0, 1], 'co': [0, 1], 'on': [1, 1], 'i': [5, 0], 'have': [2, 1], 'a': [2, 2], 'date': [1, 0], 'sunday': [1, 0], 'will': [2, 0], 'nah': [1, 0], 'dont': [2, 0], 'think': [1, 0], 'he': [2, 0], 'goes': [1, 0], 'usf': [1, 0], 'lives': [1, 0], 'around': [1, 0], 'here': [1, 0], 'though': [1, 0], 'entry': [0, 2], 'in': [1, 2], 'wkly': [0, 1], 'comp': [0, 1], 'win': [0, 2], 'fa': [0, 2], 'cup': [0, 1], 'final': [0, 1], 'tkts': [0, 1], 'st': [0, 1], 'may': [0, 1], 'text': [0, 1], 'receive': [0, 1], 'questionstd': [0, 1], 'txt': [0, 3], 'ratetcs': [0, 1], 'apply': [0, 2], 'overs': [0, 1], 'ive': [2, 0], 'been': [3, 1], 'searching': [1, 0], 'right': [1, 0], 'words': [1, 0], 'thank': 

In [14]:
palabras_ham = 0
palabras_spam = 0
palabras_diferentes = len(diccionario_contador.keys())
for key in diccionario_contador:
    palabras_ham += diccionario_contador[key][0]
    palabras_spam += diccionario_contador[key][1]
    
print(f"palabras_ham: {palabras_ham}, palabras_spam: {palabras_spam} palabras_diferentes: {palabras_diferentes}")

palabras_ham: 135, palabras_spam: 128 palabras_diferentes: 185


In [15]:
def probabilidad_palabra_dado_ham_spam(palabra):
    #Calcula la probabilidad de que una palabra esté en ham y en spam
    if palabra not in diccionario_contador:
        h = K / (palabras_ham + K * palabras_diferentes)
        s = K / (palabras_spam + K * palabras_diferentes) 
        return [h, s] # [ham, spam]
    else:
        h = (diccionario_contador[palabra][0] + K) / (palabras_ham + K * palabras_diferentes)
        s = (diccionario_contador[palabra][1] + K) / (palabras_spam + K * palabras_diferentes)
        return [h, s] # [ham, spam]

In [16]:
# Calcular las probabilidades de X_test
predicciones = []

for line in X_test:
    # calculate the probability of each word in the line
    # multiply the probabilities of each word in the line
    # if the result is greater than 0.5, add 1 to the predictions list
    # if the result is less than 0.5, add 0 to the predictions list
    probabilidad_linea_ham = probabilidad_ham # Probabilidad de que la línea sea ham
    probabilidad_linea_spam = probabilidad_spam # Probabilidad de que la línea sea spam
    for word in line.split(' '):
        valores = probabilidad_palabra_dado_ham_spam(word)
        probabilidad_linea_ham *= valores[0] 
        probabilidad_linea_spam *= valores[1]
    
    if probabilidad_linea_ham > probabilidad_linea_spam:
        predicciones.append(1)
    else:
        predicciones.append(0)
        
print(predicciones)

[1, 0, 0]


In [17]:
# Calcular la precisión
from sklearn.metrics import classification_report

print(classification_report(y_test, predicciones))

              precision    recall  f1-score   support

       False       0.50      1.00      0.67         1
        True       1.00      0.50      0.67         2

    accuracy                           0.67         3
   macro avg       0.75      0.75      0.67         3
weighted avg       0.83      0.67      0.67         3


In [18]:
from sklearn.metrics import confusion_matrix

print(confusion_matrix(y_test, predicciones))

[[1 0]
 [1 1]]


## Task 2.3 - Clasificación de mensajes futuros

In [19]:
linea = input(str("Ingrese su frase")).split(" ")
probabilidad_linea_ham = probabilidad_ham # Probabilidad de que la línea sea ham
probabilidad_linea_spam = probabilidad_spam # Probabilidad de que la línea sea spam
valores = []
for i in range(len(linea)):
    valores.append(probabilidad_palabra_dado_ham_spam(linea[i]))

print(valores)
    
for v in valores:
    probabilidad_linea_ham *= v[0] 
    probabilidad_linea_spam *= v[1]

print(f"{" ".join(linea)}: Spam {probabilidad_linea_spam} Ham: {probabilidad_linea_ham}")
    

[[0.005327650506126798, 0.0053316272126252935], [0.005327650506126798, 0.0053316272126252935]]
womp womp: Spam 1.395931857493179e-05 Ham: 1.4445357278390075e-05


In [20]:
if probabilidad_linea_ham > probabilidad_linea_spam:
    print("Es ham, con una probabilidad de: ", probabilidad_linea_ham)
else:
    print("Es spam, con una probabilidad de: ", probabilidad_linea_spam)

Es ham, con una probabilidad de:  1.4445357278390075e-05


## Task 2.4 - Comparación con Librerías

In [21]:
# Librerias Generales
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import pandas as pd

In [22]:
# Leemos el txt. 
file = open('entrenamiento.txt', 'r')
dataset = []
for line in file:
    dataset.append(line.split('    '))
dataset = pd.DataFrame(dataset) # Guardamos el data set
print(dataset.head())
dataset

      0                                                  1
0   ham  "Go until jurong point, crazy.. Available only...
1   ham                 Ok lar... Joking wif u oni...,,,\n
2  spam  Free entry in 2 a wkly comp to win FA Cup fina...
3   ham  U dun say so early hor... U c already then say...
4   ham  "Nah I don't think he goes to usf, he lives ar...


,0,1
0,ham,"""Go until jurong point, crazy.. Available only..."
1,ham,"Ok lar... Joking wif u oni...,,,\n"
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"""Nah I don't think he goes to usf, he lives ar..."
5,spam,"""FreeMsg Hey there darling it's been 3 week's ..."
6,ham,Even my brother is not like to speak with me. ...
7,ham,As per your request 'Melle Melle (Oru Minnamin...
8,spam,WINNER!! As a valued network customer you have...
9,spam,Had your mobile 11 months or more? U R entitle...


In [23]:
# Procesamiento de datos (Limpieza y estandarización)
import re
def preprocess_text(text):
    # Convierte a minúsculas
    text = text.lower()
    # Elimina caracteres no alfabéticos
    text = re.sub(r'[^a-z\s]', '', text)
    # Elimina espacios en blanco adicionales
    text = ' '.join(text.split())
    return text

In [24]:
# Aplica la función de preprocesamiento a la columna 1 del dataset
dataset[1] = dataset[1].apply(preprocess_text)

# Muestra el dataset después del preprocesamiento
dataset

,0,1
0,ham,go until jurong point crazy available only in ...
1,ham,ok lar joking wif u oni
2,spam,free entry in a wkly comp to win fa cup final ...
3,ham,u dun say so early hor u c already then say
4,ham,nah i dont think he goes to usf he lives aroun...
5,spam,freemsg hey there darling its been weeks now a...
6,ham,even my brother is not like to speak with me t...
7,ham,as per your request melle melle oru minnaminun...
8,spam,winner as a valued network customer you have b...
9,spam,had your mobile months or more u r entitled to...


In [25]:
# Separamos el dataset. 
X = dataset[1]
y = dataset[0]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=13)

In [26]:
# Vectorización de Datos
vectorizer = CountVectorizer()
X_train_vectorized = vectorizer.fit_transform(X_train)
X_test_vectorized = vectorizer.transform(X_test)

In [27]:
# Creación y entrenamiento del clasificador
alpha_value = 1.0  # Puedes ajustar este valor según tus necesidades
clf = MultinomialNB(alpha=alpha_value)
clf.fit(X_train_vectorized, y_train)

MultinomialNB()

In [28]:
# Predicciones en el conjunto de prueba
y_pred = clf.predict(X_test_vectorized)

In [29]:
# Medición de la precisión
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

Accuracy: 0.6666666666666666


In [30]:
# Probamos el modelo:
prueba = input("Introduce el texto de un correo: ")

# Preprocesamiento del texto de prueba
prueba_procesado = preprocess_text(prueba)

# Vectorización del texto de prueba
prueba_vectorizado = vectorizer.transform([prueba_procesado])

probabilidades = clf.predict_proba(prueba_vectorizado)
print("Ham \t\t Spam")
print(probabilidades)

# Hacer una predicción
prediccion = clf.predict(prueba_vectorizado)

# Mostrar la predicción
print("Predicción: ", prediccion)

Ham 		 Spam
[[0.06463847 0.93536153]]
Predicción:  ['spam']


# Comparación con librerías
- ¿Cuál implementación lo hizo mejor?
    - La implementación con librerías lo hice mejor. Fue más fácil de implementar y los márgenes en las probabilidades fueron más pronunciados. 
- ¿Por qué cree que se debe esta diferencia?
    - Las librerías de sklearn fueron creadas por expertos en la materia, que tomaron en cuenta muchos mas escenarios, facilitando el manejo de los datos proporcioandos. Sobre todo, fue más fácil la vectorización de los datos de la columna 2 (X). 

# Task 3 - Clasificación de Partidas de League of Legends
## Task 3.1 - Lectura y limpieza del dataset
- Deben hacer una breve exploración con los datos. Esto implica, pero no está limitado a:
    - Hacer encoding de las variables que se necesiten
    - Revisar si el dataset está balanceado, caso no estarlo, aplicar alguna técnica para balancearlo lo más y mejor posible
    - Escalar las variables si considera necesario
    - Selección de variables
- Recuerden hacer el split para training, testing y si consideran necesario para validation
    - 80% training
    - 20% testing
    - 10% validation si lo necesitan
- Recuerde definir de forma clara y razonada (es decir, diga el por qué de su elección) de una métrica de desempeño principal

## Task 3.2 - Support Vector Machines: Clasificación de Partidas de League of Legends

In [31]:
# puedes empezar aca
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

datos = pd.read_csv("league_of_lol.csv")

df = pd.DataFrame(datos["blueWins"])

# Pasamos la columna blueWins al final del dataset
datos = datos[[col for col in datos.columns if col != 'blueWins'] + ['blueWins']]

datos = datos.drop(['gameId'], axis = 1)

X = datos.iloc[:, :-1].values
y = datos.iloc[:, -1].values



In [32]:
datos.isnull().sum() # No hay valores nulos

blueWardsPlaced                 0
blueWardsDestroyed              0
blueFirstBlood                  0
blueKills                       0
blueDeaths                      0
blueAssists                     0
blueEliteMonsters               0
blueDragons                     0
blueHeralds                     0
blueTowersDestroyed             0
blueTotalGold                   0
blueAvgLevel                    0
blueTotalExperience             0
blueTotalMinionsKilled          0
blueTotalJungleMinionsKilled    0
blueGoldDiff                    0
blueExperienceDiff              0
blueCSPerMin                    0
blueGoldPerMin                  0
redWardsPlaced                  0
redWardsDestroyed               0
redFirstBlood                   0
redKills                        0
redDeaths                       0
redAssists                      0
redEliteMonsters                0
redDragons                      0
redHeralds                      0
redTowersDestroyed              0
redTotalGold  

In [33]:
df['blueWins'].value_counts()

blueWins
0    4949
1    4930
Name: count, dtype: int64

In [34]:

#Dividir el conjunto de datos en entrenamiento y prueba
from sklearn.model_selection import train_test_split
X_entreno, X_prueba, y_entreno, y_prueba = train_test_split(X, y, test_size = 0.2, random_state = 1)

# Escalar variables
from sklearn.preprocessing import MinMaxScaler
escalador = MinMaxScaler()
X_entreno[:, :-1] = escalador.fit_transform(X_entreno[:, :-1])
X_prueba[:, :-1] = escalador.fit_transform(X_prueba[:, :-1])


# Selección de variables
# Decidimos quitar las siguientes variables ya que por nuestro conocimiento en LOL es lo que hemos sentido que no 
# influye en ganar o no una partida.
#blueTotalGold, blueTotalExperience, blueGoldDiff,blueExperienceDiff, blueCSPerMin, blueGoldPerMin 

In [35]:
#redTotalGold, redTotalExperience, redGoldDiff,redExperienceDiff, redCSPerMin, redGoldPerMin 

datos = datos.drop(['blueTotalGold', "blueTotalExperience", "blueGoldDiff",
                    "blueExperienceDiff","blueCSPerMin",
                    "blueGoldPerMin", "redTotalGold", "redTotalExperience", 
                    "redGoldDiff","redExperienceDiff", "redCSPerMin", "redGoldPerMin"], axis = 1)



